<a href="https://colab.research.google.com/github/288756/VisArtificial/blob/master/Miniproyecto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
from google.colab import drive
from sklearn.preprocessing import LabelEncoder
import os
from PIL import Image
import tensorflow as tf
from tensorflow.data import Dataset
import matplotlib.pyplot as plt
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input
import numpy as np
import tensorflow_addons as tfa
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Montar el Google Drive en el directorio del proyecto y descomprimir el fichero con los datos
drive.mount('/content/gdrive')
!unzip -n '/content/gdrive/My Drive/vision-artificial.zip' >> /dev/null  # ACTUALIZAR: ruta al fichero comprimido


/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


Mounted at /content/gdrive


In [2]:
!pip install tensorflow_addons

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 3.7 MB/s eta 0:00:00


In [4]:
# Especificar las rutas al directorio con las imágenes y al fichero con las etiquetas
imgtrain_dir = "/content/train/images/"
imgtrainmask_dir = "/content/train/masks/"
csvtrain_file = "/content/train.csv"

imgtest_dir = "/content/train.csv"
csvtest_file = "/content/test.csv"
# Leer el fichero CSV con las etiquetas
df = pd.read_csv(csvtrain_file, dtype={"class": "category"})
dftest = pd.read_csv(csvtest_file, dtype={"class": "category"})

# Codificar las etiquetas utilizando LabelEncoder
label_encoder = LabelEncoder()
df['class_encoded'] = label_encoder.fit_transform(df['class'])

# Convertir las etiquetas codificadas en un vector one-hot
class_one_hot = pd.get_dummies(df['class_encoded'], prefix='class')

# Renombrar las columnas del vector one-hot
class_one_hot.columns = ['normal', 'benign', 'malignant']

# Concatenar el DataFrame original con las etiquetas one-hot y la columna 'mask_filename'
df = pd.concat([df[['image_filename', 'mask_filename']], class_one_hot], axis=1)

In [5]:
column_names = df.columns.tolist()
print(df)
print(column_names[2:5])

    image_filename mask_filename  normal  benign  malignant
0    image_003.png  mask_003.png    True   False      False
1    image_004.png  mask_004.png    True   False      False
2    image_008.png  mask_008.png    True   False      False
3    image_012.png  mask_012.png    True   False      False
4    image_016.png  mask_016.png    True   False      False
..             ...           ...     ...     ...        ...
332  image_399.png  mask_399.png   False   False       True
333  image_400.png  mask_400.png   False   False       True
334  image_416.png  mask_416.png   False   False       True
335  image_419.png  mask_419.png   False   False       True
336  image_443.png  mask_443.png   False   False       True

[337 rows x 5 columns]
['normal', 'benign', 'malignant']


In [6]:
# Dividir el conjunto en entrenamiento, validación y test (80:10:10)
val_size = int(len(df) * 0.15)
test_size = int(len(df) * 0.15)

df = df.sample(frac=1).reset_index(drop=True)  # barajar el dataframe
dftest = df[:test_size]
dfval = df[test_size:test_size+val_size]
dftrain = df[test_size+val_size:]

print(f'Número de ejemplos del conjunto de entrenamiento: {dftrain.shape[0]}')
print(f'Número de ejemplos del conjunto de validación: {dfval.shape[0]}')
print(f'Número de ejemplos del conjunto de test: {dftest.shape[0]}')
dftrain = dftrain.reset_index(drop=True)
dfval = dfval.reset_index(drop=True)
dftest = dftest.reset_index(drop=True)

Número de ejemplos del conjunto de entrenamiento: 237
Número de ejemplos del conjunto de validación: 50
Número de ejemplos del conjunto de test: 50


In [7]:
dftrain

,image_filename,mask_filename,normal,benign,malignant
0,image_138.png,mask_138.png,False,False,True
1,image_121.png,mask_121.png,False,False,True
2,image_391.png,mask_391.png,True,False,False
3,image_047.png,mask_047.png,False,True,False
4,image_031.png,mask_031.png,True,False,False
...,...,...,...,...,...
232,image_371.png,mask_371.png,True,False,False
233,image_448.png,mask_448.png,False,True,False
234,image_256.png,mask_256.png,False,True,False
235,image_401.png,mask_401.png,False,True,False


In [8]:
archivos = os.listdir(imgtrain_dir)
anchuras = []
alturas = []
for archivo in archivos:
      imagen = Image.open(os.path.join(imgtrain_dir, archivo))
      ancho, alto = imagen.size
      anchuras.append(ancho)
      alturas.append(alto)
media_anchura = sum(anchuras) / len(anchuras)
media_altura = sum(alturas) / len(alturas)

print("Media de anchura:", media_anchura)
print("Media de altura:", media_altura)

Media de anchura: 634.9139465875371
Media de altura: 514.9317507418398


In [10]:
# Dimensiones deseadas de la imagen
img_width, img_height = 634,514
n_channels = 3                # número de canales (RGB)
n_classes = 3                 # número de clases
x_col = 'image_filename'      # nombres de las columnas en el fichero CSV
y_col = ['normal', 'benign', 'malignant']  # lista de nombres de las columnas de las etiquetas

def load_and_preprocess_image(image_filename, label):
    image_path = tf.strings.join([imgtrain_dir, image_filename])
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=n_channels)

    # Aumento de datos: rotación aleatoria
    # image = tfa.image.rotate(image, tf.random.uniform(shape=[], minval=-5, maxval=5))

    image = tf.image.resize(image, [img_width, img_height])
    image = image / 255.0
    return image, label

# Crear conjunto de datos
def get_dataset(df):
    image_paths = df[x_col].values
    labels = df[y_col].values
    dataset = Dataset.from_tensor_slices((image_paths, labels))
    dataset = dataset.map(load_and_preprocess_image)
    return dataset

# Crear los conjuntos de datos y preparar los lotes
batch_size = 32
train_dataset = get_dataset(dftrain).batch(batch_size)
val_dataset = get_dataset(dfval).batch(batch_size)
test_dataset = get_dataset(dftest).batch(batch_size)

print(f'Número de lotes del conjunto de entrenamiento: {len(train_dataset)}')
print(f'Número de lotes del conjunto de validación: {len(val_dataset)}')
print(f'Número de lotes del conjunto de test: {len(test_dataset)}')

Número de lotes del conjunto de entrenamiento: 8
Número de lotes del conjunto de validación: 2
Número de lotes del conjunto de test: 2


In [15]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras import backend as K

from keras.layers import Input, Conv2D, MaxPooling2D, concatenate, Flatten, Dense, AveragePooling2D, BatchNormalization, Dropout
from keras.models import Model

def get_model(input_shape, num_classes):
    # Input layer
    input_layer = Input(shape=input_shape)

    # First convolutional layer
    x = Conv2D(64, (7, 7), strides=(2, 2), padding='same', activation='relu')(input_layer)
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
    x = BatchNormalization()(x)

    # Define Inception module
    def inception_module(prev_layer, filters):
        conv1x1 = Conv2D(filters[0], (1, 1), padding='same', activation='relu')(prev_layer)
        conv3x3 = Conv2D(filters[1], (3, 3), padding='same', activation='relu')(prev_layer)
        conv5x5 = Conv2D(filters[2], (5, 5), padding='same', activation='relu')(prev_layer)
        pool = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(prev_layer)
        pool_conv = Conv2D(filters[3], (1, 1), padding='same', activation='relu')(pool)
        output = concatenate([conv1x1, conv3x3, conv5x5, pool_conv], axis=-1)
        return output

    # First inception module
    x = inception_module(x, [64, 128, 32, 32])
    x = BatchNormalization()(x)

    # Second inception module
    x = inception_module(x, [128, 192, 96, 64])
    x = BatchNormalization()(x)

    # Third inception module
    x = inception_module(x, [192, 256, 128, 64])
    x = BatchNormalization()(x)

    # Fourth inception module
    x = inception_module(x, [256, 320, 160, 128])
    x = BatchNormalization()(x)

    # Flatten
    x = AveragePooling2D((5, 5), strides=(3, 3))(x)
    x = Conv2D(128, (1, 1), padding='same', activation='relu')(x)
    x = Flatten()(x)

    # Fully connected layers
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    output_layer = Dense(num_classes, activation='softmax')(x)

    # Create model
    model = Model(input_layer, output_layer)

    return model
def f1_metric(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    precision = tp / (tp + fp + K.epsilon())
    recall = tp / (tp + fn + K.epsilon())

    f1 = 2 * precision * recall / (precision + recall + K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)

    # Compute the weighted F1 score
    weights = K.sum(y_true, axis=0) / K.sum(y_true)
    weighted_f1 = K.sum(weights * f1)

    return weighted_f1

# Definir el tamaño de entrada y el número de clases
input_shape = (img_width, img_height, n_channels)
num_classes = 3  # Normal, Benigno, Maligno
model = get_model(input_shape,num_classes)

model.compile(loss='categorical_crossentropy',  # función de pérdida para problemas de clasificación multi-clase
              optimizer="adam",
              metrics=[f1_metric])

# Imprimir el resumen del modelo
model.summary()


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 634, 514, 3)]        0         []                            
                                                                                                  
 conv2d_11 (Conv2D)          (None, 317, 257, 64)         9472      ['input_3[0][0]']             
                                                                                                  
 max_pooling2d_4 (MaxPoolin  (None, 159, 129, 64)         0         ['conv2d_11[0][0]']           
 g2D)                                                                                             
                                                                                                  
 batch_normalization (Batch  (None, 159, 129, 64)         256       ['max_pooling2d_4[0][0]'

In [16]:
early_stopping = EarlyStopping(monitor='val_accuracy', patience=20, mode='max')
# reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.2, patience=5, min_lr=0.00001) Pocas épocas para que se note

model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=30,
    verbose = 2,
    steps_per_epoch=np.ceil(len(dftrain) / batch_size),
    validation_steps=np.ceil(len(dfval) / batch_size),
    callbacks=[early_stopping]#, reduce_lr]
)


Epoch 1/30


ResourceExhaustedError: Graph execution error:

Detected at node model_1/batch_normalization_2/FusedBatchNormV3 defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.10/dist-packages/tornado/ioloop.py", line 685, in <lambda>

  F

In [13]:
test_loss, test_medida = model.evaluate(test_dataset, verbose=2)
print("test_loss: %.4f, test_medida: %.4f" % (test_loss, test_medida))

2/2 - 1s - loss: 1.3035 - f1_metric: 0.6139 - 673ms/epoch - 336ms/step
test_loss: 1.3035, test_medida: 0.6139


In [ ]:
# Directorio donde se encuentran las imágenes de prueba
test_images_dir = 'test/images/'

# Obtener la lista de nombres de archivos de las imágenes de prueba
test_filenames = os.listdir(test_images_dir)

# Crear un DataFrame para almacenar las predicciones
predictions_df = pd.DataFrame({'image_filename': test_filenames})

# Crear columnas para 'normal', 'benign' y 'malignant' con valores iniciales de 0
predictions_df['normal'] = 0
predictions_df['benign'] = 0
predictions_df['malignant'] = 0

# Iterar sobre cada imagen de prueba
for filename in test_filenames:
    # Cargar la imagen y preprocesarla
    img_path = os.path.join(test_images_dir, filename)
    img = load_img(img_path, target_size=(img_width, img_height))
    img_array = img_to_array(img)
    img_array = preprocess_input(img_array)
    img_array = np.expand_dims(img_array, axis=0)  # Añadir una dimensión adicional para el lote

    # Realizar la predicción
    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction)  # Obtener la clase predicha

    # Actualizar las columnas correspondientes según la predicción
    if predicted_class == 0:
        predictions_df.loc[predictions_df['image_filename'] == filename, 'normal'] = 1
    elif predicted_class == 1:
        predictions_df.loc[predictions_df['image_filename'] == filename, 'benign'] = 1
    elif predicted_class == 2:
        predictions_df.loc[predictions_df['image_filename'] == filename, 'malignant'] = 1

# Ordenar el DataFrame por el nombre del archivo de imagen
predictions_df = predictions_df.sort_values(by='image_filename')

# Guardar el DataFrame en un archivo CSV
predictions_df.to_csv('test_predictions.csv', index=False)

# Imprimir las primeras filas del DataFrame para verificar
print(predictions_df.head())


1/1 [==============================] - 0s 22ms/step
    image_filename  normal  benign  malignant
64   image_005.png       1       0          0
53   image_006.png       0       1          0
112  image_007.png       1       0          0
40   image_009.png       1       0          0
21   image_010.png       1       0          0
